# Setup

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from skimage.metrics import structural_similarity as ssim
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Dataset and Models

## Datasets

In [2]:
def load_images_and_shadow_maps(images_dir, shadow_maps_dir, image_size=(256, 256)):
    images = []
    shadow_maps = []
    for img_name in os.listdir(images_dir):
        img_path = os.path.join(images_dir, img_name)
        shadow_map_path = os.path.join(shadow_maps_dir, img_name)

        # Load and preprocess images
        image = load_img(img_path, target_size=image_size)
        image = img_to_array(image) / 255.0  # Normalize

        # Load and preprocess shadow maps
        shadow_map = load_img(shadow_map_path, color_mode='grayscale', target_size=image_size)
        shadow_map = img_to_array(shadow_map) / 255.0  # Normalize

        images.append(image)
        shadow_maps.append(shadow_map)

    return np.array(images), np.array(shadow_maps)

# Load training data
train_images_dir = '/content/drive/MyDrive/Capstone/ISTD/data/train/images'
train_shadow_maps_dir = '/content/drive/MyDrive/Capstone/ISTD/data/train/shadow_maps'
x_train, y_train = load_images_and_shadow_maps(train_images_dir, train_shadow_maps_dir)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Load test data
test_images_dir = '/content/drive/MyDrive/Capstone/ISTD/data/test/images'
test_shadow_maps_dir = '/content/drive/MyDrive/Capstone/ISTD/data/test/shadow_maps'
x_test, y_test = load_images_and_shadow_maps(test_images_dir, test_shadow_maps_dir)

print(f'Train images shape: {x_train.shape}, Train shadow maps shape: {y_train.shape}')
print(f'Validation images shape: {x_val.shape}, Validation shadow maps shape: {y_val.shape}')
print(f'Test images shape: {x_test.shape}, Test shadow maps shape: {y_test.shape}')

Train images shape: (1064, 256, 256, 3), Train shadow maps shape: (1064, 256, 256, 1)
Validation images shape: (266, 256, 256, 3), Validation shadow maps shape: (266, 256, 256, 1)
Test images shape: (540, 256, 256, 3), Test shadow maps shape: (540, 256, 256, 1)


## Models

In [3]:
# Baseline CNN model for comparison
def simple_model(input_shape):
    inputs = Input(input_shape)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D((2, 2))(conv1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    pool3 = MaxPooling2D((2, 2))(conv3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    pool4 = MaxPooling2D((2, 2))(conv4)
    up1 = UpSampling2D((2, 2))(pool4)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
    up2 = UpSampling2D((2, 2))(conv5)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
    up3 = UpSampling2D((2, 2))(conv6)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
    up4 = UpSampling2D((2, 2))(conv7)
    conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(up4)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv8)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [17]:
# VGG16-based Model
def vgg16_model(input_shape=(256, 256, 3)):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    inputs = Input(input_shape)
    x = base_model(inputs)
    conv = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    up1 = UpSampling2D((2, 2))(conv)
    conv1 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
    up2 = UpSampling2D((2, 2))(conv1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
    up3 = UpSampling2D((2, 2))(conv2)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
    up4 = UpSampling2D((2, 2))(conv3)
    conv4 = Conv2D(32, (3, 3), activation='relu', padding='same')(up4)
    up5 = UpSampling2D((2, 2))(conv4)
    conv5 = Conv2D(16, (3, 3), activation='relu', padding='same')(up5)
    outputs = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(conv5)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [18]:
# Define ResNet50-based model
def resnet50_model(input_shape=(256, 256, 3)):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    inputs = Input(input_shape)
    x = base_model(inputs)
    conv = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    up1 = UpSampling2D((2, 2))(conv)
    conv1 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
    up2 = UpSampling2D((2, 2))(conv1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
    up3 = UpSampling2D((2, 2))(conv2)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
    up4 = UpSampling2D((2, 2))(conv3)
    conv4 = Conv2D(32, (3, 3), activation='relu', padding='same')(up4)
    up5 = UpSampling2D((2, 2))(conv4)
    conv5 = Conv2D(16, (3, 3), activation='relu', padding='same')(up5)
    outputs = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(conv5)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Training and Evaluation

In [19]:
# Evaluation
def compute_metrics(true, pred):
    mse = np.mean((true - pred) ** 2)
    mae = np.mean(np.abs(true - pred))
    ssim_value = ssim(true.squeeze(), pred.squeeze(), data_range=pred.max() - pred.min())
    return mse, mae, ssim_value

In [20]:
simple_model = simple_model((256, 256, 3))
vgg16_based_model = vgg16_model((256, 256, 3))
resnet50_based_model = resnet50_model((256, 256, 3))

simple_model.compile(optimizer=Adam(), loss=MeanSquaredError(), metrics=[MeanAbsoluteError()])
vgg16_based_model.compile(optimizer=Adam(), loss=MeanSquaredError(), metrics=[MeanAbsoluteError()])
resnet50_based_model.compile(optimizer=Adam(), loss=MeanSquaredError(), metrics=[MeanAbsoluteError()])

In [21]:
# Training
callbacks = [
    EarlyStopping(patience=2, restore_best_weights=True),
    ModelCheckpoint("model.h5", save_best_only=True)
]

simple_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=3, batch_size=32, callbacks=callbacks)
vgg16_based_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=3, batch_size=32, callbacks=callbacks)
resnet50_based_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=3, batch_size=32, callbacks=callbacks)

Epoch 1/3
34/34 [==============================] - ETA: 0s - loss: 0.1586 - mean_absolute_error: 0.2815

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


34/34 [==============================] - 43s 671ms/step - loss: 0.1586 - mean_absolute_error: 0.2815 - val_loss: 0.1262 - val_mean_absolute_error: 0.2581
Epoch 2/3
34/34 [==============================] - 12s 356ms/step - loss: 0.1174 - mean_absolute_error: 0.2353 - val_loss: 0.1120 - val_mean_absolute_error: 0.1974
Epoch 3/3
34/34 [==============================] - 11s 329ms/step - loss: 0.1056 - mean_absolute_error: 0.2081 - val_loss: 0.1029 - val_mean_absolute_error: 0.2234
Epoch 1/3
34/34 [==============================] - 25s 489ms/step - loss: 0.1992 - mean_absolute_error: 0.2071 - val_loss: 0.1956 - val_mean_absolute_error: 0.1957
Epoch 2/3
34/34 [==============================] - 9s 269ms/step - loss: 0.1975 - mean_absolute_error: 0.1975 - val_loss: 0.1956 - val_mean_absolute_error: 0.1957
Epoch 3/3
34/34 [==============================] - 9s 279ms/step - loss: 0.1975 - mean_absolute_error: 0.1975 - val_loss: 0.1956 - val_mean_absolute_error: 0.1957


In [22]:
# Save models
simple_model.save('simple_cnn_model.h5')
vgg16_based_model.save('vgg16_based_model.h5')
resnet50_based_model.save('resnet50_based_model.h5')

In [24]:
# Evaluate models
y_pred_simple = simple_model.predict(x_test)
y_pred_vgg16 = vgg16_based_model.predict(x_test)
y_pred_resnet50 = resnet50_based_model.predict(x_test)

mse_cnn, mae_cnn, ssim_cnn = compute_metrics(y_test, y_pred_simple)
mse_vgg16, mae_vgg16, ssim_vgg16 = compute_metrics(y_test, y_pred_vgg16)
mse_resnet50, mae_resnet50, ssim_resnet50 = compute_metrics(y_test, y_pred_resnet50)

print(f"VGG16-based Model - MSE: {mse_vgg16}, MAE: {mae_vgg16}, SSIM: {ssim_vgg16}")
print(f"ResNet50-based Model - MSE: {mse_resnet50}, MAE: {mae_resnet50}, SSIM: {ssim_resnet50}")
print(f"Baseline Model - MSE: {mse_cnn}, MAE: {mae_cnn}, SSIM: {ssim_cnn}")

17/17 [==============================] - 6s 296ms/step
VGG16-based Model - MSE: 0.11255288124084473, MAE: 0.24572539329528809, SSIM: 0.20753137231790972
ResNet50-based Model - MSE: 0.15776506066322327, MAE: 0.1577674001455307, SSIM: nan
Baseline Model - MSE: 0.06310500204563141, MAE: 0.10150318592786789, SSIM: 0.5488462264802839


/usr/local/lib/python3.10/dist-packages/skimage/metrics/_structural_similarity.py:230: RuntimeWarning: invalid value encountered in divide
  S = (A1 * A2) / D
